In [4]:
%run "DataPreparation.ipynb"
%run "Modeling.ipynb"

(None, 512, 512, 32)


In [5]:
# Setup
#directory_path = "../Data"
# images_dir= "C:/Users/sowmi/Python scripts/hurricane/images" #images
# json_dir = "C:/Users/sowmi/Python scripts/hurricane/labels" #json
images_dir= "../Images/hurricane" #images
json_dir = "../Data/hurricane" #json

# Loading JSON data into dataframe
hurricane_pre_df, hurricane_post_df, pre_hurricane_images, post_hurricane_images = prepare_data(images_dir, json_dir);

Retrieving pre and post disaster images from: ../Images/hurricane

Total pre-disaster images: 1219

Total post-disaster images: 1219


In [6]:
pre_resized_img, pre_mask_resized_img= preprocess_data(hurricane_pre_df, images_dir);

Completed saving mask images to the ..\masks directory
Total pre-disaster mask images: 1219


In [7]:
X_train, X_valid, y_train, y_valid=split_data(pre_resized_img, pre_mask_resized_img);

(1219, 256, 256, 3)
(1219, 256, 256, 1)


In [8]:
#ONLY IF TRAINING IS NEEDED!
# train_fcn_model();
# train_unet_model();

In [25]:
fcn_model = tf.keras.models.load_model('model/FCN/FCN_model.keras')

fcn_model.summary()
# Making predictions
fcn_model_mask = fcn_model.predict(X_valid)
# Model Evaluation
loss, accuracy = fcn_model.evaluate(X_valid, y_valid)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

TypeError: Error when deserializing class 'Conv2DTranspose' using config={'name': 'conv2d_transpose_6', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': [3, 3], 'strides': [2, 2], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'groups': 1, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None, 'output_padding': None}.

Exception encountered: Unrecognized keyword arguments passed to Conv2DTranspose: {'groups': 1}

# Unet Model

In [ ]:
unet_model = tf.keras.models.load_model('model/Unet/Unet_model.keras')
unet_model.summary()
# Making predictions
unet_model_mask = unet_model.predict(X_valid)
# Model Evaluation
loss, accuracy = unet_model.evaluate(X_valid, y_valid)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")